In [1]:
from utils import *
from models import *
from qd_data import *
import keras
import numpy as np
from keras.applications.mobilenet_v2 import MobileNetV2
from imgaug import augmenters as iaa
import imgaug as ia
%matplotlib inline

Using TensorFlow backend.


### Load Data

In [2]:
# df_all = pd.read_csv(data_path+"df_all_raw.csv")


In [3]:
from keras.applications.resnet50 import ResNet50


In [4]:
classfiles = glob(train_path+"*.csv")
nametoid = {v[:-4].split("/")[-1].replace(" ", "_") :i
               for i, v in enumerate(classfiles) if "df_all_raw" not in v} #adds underscores
idtoname = {i: v[:-4].split("/")[-1].replace(" ", "_")
               for i, v in enumerate(classfiles) if "df_all_raw" not in v} #adds underscores

imsize=128  
batch_size=128

In [16]:
models = {"resnet50_":keras.applications.resnet50.ResNet50(input_shape=(imsize, imsize, 1), weights=None, classes=num_classes)},
{"inception_resnet_v2_":keras.applications.InceptionResNetV2(input_shape=(imsize, imsize, 1),  weights=None, classes=num_classes)}


In [25]:
models[0].items()[0]

TypeError: 'dict_items' object does not support indexing

In [4]:
small_df = pd.read_csv("../data/small-set.csv")




In [5]:
exp1.train(MobileNetV2(input_shape=(32, 32, 1), alpha=1., weights=None, classes=num_classes),
          small_df,small_df,continue_training=True)

Epoch 1/2
1/1 [==============================] - 6s 6s/step - loss: 6.0744 - acc: 0.0000e+00 - top_3_accuracy: 0.0000e+00 - val_loss: 14.9166 - val_acc: 0.0000e+00 - val_top_3_accuracy: 0.0000e+00
Epoch 2/2
1/1 [==============================] - 0s 73ms/step - loss: 6.1668 - acc: 0.0000e+00 - top_3_accuracy: 0.0000e+00 - val_loss: 12.5090 - val_acc: 0.0000e+00 - val_top_3_accuracy: 0.0000e+00


In [29]:
# _10fold0 = load_obj("../data/folds10.pik")[0]
# train_index = _10fold0["train_index"]
valid_index = _10fold0["test_index"]

In [31]:
df_all.iloc[valid_index].to_csv("../data/valid_df.csv",index=False)

In [4]:
sample_df = pd.read_csv("../data/sample_train_df_half.csv")

In [8]:
valid_df = pd.read_csv("../data/valid_df.csv")

In [5]:
small_df = pd.read_csv("../data/small-set.csv")

In [148]:
valid_df = small_df

Epoch 1/2
  1/277 [..............................] - ETA: 1:49:45 - loss: 5.9165 - acc: 0.0000e+00 - top_3_accuracy: 0.0000e+00

Process ForkPoolWorker-9:
Process ForkPoolWorker-12:
Process ForkPoolWorker-10:
Process ForkPoolWorker-7:
Process ForkPoolWorker-11:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-8:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/tt/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/tt/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda3/envs/tt/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda3/envs/tt/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/tt/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._ar

Epoch 1/2


  File "/home/ubuntu/anaconda3/envs/tt/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/ubuntu/anaconda3/envs/tt/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
KeyboardInterrupt
  File "/home/ubuntu/anaconda3/envs/tt/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: 

In [6]:
# train_gen = QD_Datagen(df_all.drawing.iloc[train_index],df_all.target.iloc[train_index],4, is_train_mode = True)
train_gen = QD_Datagen(sample_df.drawing,
                       sample_df.target,
                       batch_size,
                       is_train_mode = True, imsize=imsize)


In [44]:
valid_gen = QD_Datagen(valid_df.drawing,
                       valid_df.target,
                       batch_size, 
                       is_train_mode=False,
                       imsize=imsize)

### Model

In [3]:
exp_name = "xception_augv2_im128_fold5_nonrecxxx"
# callbacks = get_callbacks(exp_name, batch_size=batch_size)


In [18]:
model = keras.applications.InceptionResNetV2(input_shape=(imsize, imsize, 1), weights=None, classes=num_classes)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', top_3_accuracy])

NameError: name 'top_k_categorical_accuracy' is not defined

In [6]:
model.load_weights(f"../weights/{exp_name}.hdf5")

In [11]:
hist = model.fit_generator(valid_gen,
                           steps_per_epoch=10,#len(valid_gen)//(batch_size*10000),
                           epochs = 2,
                           validation_data = valid_gen,
                           validation_steps=10,  
                           callbacks = callbacks,
                           verbose = 1)

Epoch 1/2
 9/10 [==========================>...] - ETA: 1s - loss: 6.3528 - acc: 0.0000e+00 - top_3_accuracy: 0.0000e+00WARNING:tensorflow:This model was compiled with a Keras optimizer (<tensorflow.python.keras.optimizers.Adam object at 0x7fc67e0bd7f0>) but is being saved in TensorFlow format with `save_weights`. The model's weights will be saved, but unlike with TensorFlow optimizers in the TensorFlow format the optimizer's state will not be saved.

Consider using a TensorFlow optimizer from `tf.train`.
10/10 [==============================] - 19s 2s/step - loss: 6.4495 - acc: 0.0000e+00 - top_3_accuracy: 0.0000e+00 - val_loss: 5.8291 - val_acc: 0.0000e+00 - val_top_3_accuracy: 0.0000e+00
Epoch 2/2
10/10 [==============================] - 1s 81ms/step - loss: 7.3167 - acc: 0.0000e+00 - top_3_accuracy: 0.0250 - val_loss: 5.8293 - val_acc: 0.0000e+00 - val_top_3_accuracy: 0.0000e+00


### Test prediction

In [5]:
batch_size=128
imsize=128

In [6]:
exp_name = "xception_augv2_im128_fold5_nonrecxxx"

In [9]:
test_df = pd.read_csv('../data/test_simplified.csv')
test_df.shape

(112199, 3)

In [15]:
test_df.drawing.shape

(112199,)

In [6]:
test_gen = QD_Datagen(test_df.drawing.values,
                       np.zeros(test_df.shape[0]),
                       batch_size, 
                       is_train_mode=False,
                       imsize=imsize)

NameError: name 'batch_size' is not defined

In [30]:
test_predictions = model.predict_generator(test_gen, steps=np.ceil(len(test_df)/batch_size), verbose=1)
test_predictions = test_predictions[:test_df.shape[0]]

877/877 [==============================] - 81s 92ms/step


In [7]:
exp = Experiment(imsize,batch_size,exp_name)

In [8]:
exp.predict(num_augmentations=5)

Predicting using model found in /home/ubuntu/draw/weights/xception_augv2_im128_fold5_nonrecxxx.hdf5 ..
877/877 [==============================] - 273s 311ms/step


In [17]:
exp_name

'xception_augv2_im128_fold5_nonrecxxx'

In [18]:
test_predictions = np.load(f"../predictions/{exp_name}.npy")

In [13]:
test_predictions = [np.load("../predictions/denseNet121_im128_fold8_nonrecx.npy"),
                    np.load("../predictions/xception_augv2_im128_fold5_nonrecxxx_augx5.npy"),
                    ]
                            

In [14]:
test_predictions = np.mean(test_predictions,axis=0)

In [15]:
test_top3_id = preds2catids(test_predictions)
top3cats = test_top3_id.replace(idtoname)


In [16]:
top3cats.shape, test_df.shape

((112199, 3), (112199, 4))

In [17]:
top3cats.a.value_counts()

coffee_cup                 630
calendar                   601
tornado                    551
cake                       525
teddy-bear                 461
peas                       441
bus                        435
brain                      431
grapes                     428
train                      418
paintbrush                 414
raccoon                    403
pig                        403
toe                        401
goatee                     397
camouflage                 396
ambulance                  395
cell_phone                 393
knee                       392
barn                       391
golf_club                  390
face                       388
monkey                     387
leaf                       385
bulldozer                  385
The_Great_Wall_of_China    384
shorts                     384
garden                     384
firetruck                  382
belt                       382
                          ... 
car                        268
bat     

In [18]:
test_df['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test_df[['key_id', 'word']]
submission.to_csv('/home/ubuntu/draw/submissions/{}.csv'.format("ens_xcp_dens121"), index=False)